In [99]:
# Ask shapereader to find the world regions

import pandas as pd
import cartopy.io.shapereader as shpreader

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) # GEOPANDAS DF

shpfilename = shpreader.natural_earth(resolution='110m', 
                                      category='cultural',
                                      name='admin_0_countries')

In [67]:
reader = shpreader.Reader(shpfilename)
countries = reader.records()
country = next(countries)

def nome(s):
    df = s.attributes['NAME_EN'], s.attributes['REGION_WB'], s.attributes['NAME'], 
    s.attributes['GEOUNIT'], s.attributes['GEOU_DIF'],  s.attributes['GU_A3'],
    s.attributes['BRK_GROUP'] , s.attributes['POSTAL'] 
    return df

def region(c):
    countries_bn = sorted(countries, key=c)
    name_reg = []
    for country in countries_bn:
        name_reg.append([country.attributes['NAME_EN'], str(country.attributes['REGION_WB']), str(country.attributes['REGION_UN'])])
    df = pd.DataFrame(name_reg, columns=['name', 'regionw', 'regionun'])
    df = df.reset_index(drop=True)
    return(df)

region = region(nome)

In [106]:
rworld = world.merge(region, on='name', how='left', indicator=True)

rworld_miss = rworld[rworld._merge != 'both']
rworld_miss.astype({'geometry': str}).to_csv(r'/home/hmdg/Documents/Data_Science/Repos/Lands/data/rworld_miss.csv', index = False)

In [107]:
rworld_miss_mod = pd.read_csv('/home/hmdg/Documents/Data_Science/Repos/Lands/data/rworld_miss_mod.csv', 
                      header=0, encoding='utf-8-sig')#'iso8859_1')#'

In [111]:
rworld_miss_mod = pd.DataFrame(rworld_miss_mod)
rworld_both = rworld[rworld._merge == 'both']

# DF with both data region, continent and country
reg_world= pd.concat([rworld_both, rworld_miss_mod]).reset_index(drop=True)

reg_world= reg_world.drop(columns=['_merge'])

reg_world.astype({'geometry': str}).to_csv(r'/home/hmdg/Documents/Data_Science/Repos/Lands/data/reg_world.csv', index = False)

In [94]:
region.shape, world.shape, rworld.shape, reg_world.shape

((176, 3), (177, 6), (177, 9), (177, 9))

In [110]:
reg_world.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 8 columns):
pop_est       177 non-null int64
continent     177 non-null object
name          177 non-null object
iso_a3        177 non-null object
gdp_md_est    177 non-null float64
geometry      177 non-null object
regionw       177 non-null object
regionun      157 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 11.1+ KB
